# Example of the warm up
We will try to understand Valen's Warm up and its associated loss Parial_Loss

In [1]:
import torch
import torch.nn as nn
import numpy as np

from datasets.openml_datasets import OpenML_Dataset
from datasets.torch_datasets import Torch_Dataset
from utils.weakener import Weakener
from models.model import MLP
from utils.losses import PartialLoss
from utils.trainig_testing import train_and_evaluate,warm_up


In [2]:
Data = Torch_Dataset('mnist',batch_size=8)
train_x, train_y, test_x, test_y = Data.get_data()

In [3]:
Weak = Weakener(Data.num_classes)
Weak.generate_M(model_class='pll')

In [4]:
Weak.M,Weak.Z

(array([[0.        , 0.        , 0.        , ..., 0.        , 0.00217014,
         0.00217014],
        [0.        , 0.        , 0.        , ..., 0.00217014, 0.        ,
         0.00217014],
        [0.        , 0.        , 0.        , ..., 0.00217014, 0.00217014,
         0.        ],
        ...,
        [0.00195312, 0.00195312, 0.00195312, ..., 0.00195312, 0.        ,
         0.00195312],
        [0.00195312, 0.00195312, 0.00195312, ..., 0.00195312, 0.00195312,
         0.        ],
        [0.00195312, 0.00195312, 0.00195312, ..., 0.00195312, 0.00195312,
         0.00195312]]),
 array([[0, 0, 0, ..., 0, 1, 1],
        [0, 0, 0, ..., 1, 0, 1],
        [0, 0, 0, ..., 1, 1, 0],
        ...,
        [1, 1, 1, ..., 1, 0, 1],
        [1, 1, 1, ..., 1, 1, 0],
        [1, 1, 1, ..., 1, 1, 1]]))

In [5]:
z,w = Weak.generate_weak(train_y)

In [6]:
train_y[:5,:],z[:5],w[:5,:]

(tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
 tensor([ 79, 937, 474, 303, 554], dtype=torch.int32),
 tensor([[0., 0., 0., 1., 0., 1., 0., 1., 1., 1.],
         [1., 1., 1., 0., 1., 1., 0., 1., 0., 0.],
         [0., 1., 1., 1., 1., 0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 1., 1., 1., 0., 0., 1.],
         [1., 0., 0., 0., 1., 1., 0., 1., 0., 1.]], dtype=torch.float64))

In [7]:
train_y.shape,z.shape,w.shape

(torch.Size([60000, 10]), torch.Size([60000]), torch.Size([60000, 10]))

In [8]:
int(Weak.c)

10

In [9]:
Weak.generate_wl_priors()

C:\Users\danibacaicoa\Anaconda3\envs\Weak_Label_Learning\lib\site-packages\cvxpy\problems\problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


array([0.00041072, 0.00045548, 0.00045243, ..., 0.00176967, 0.00176692,
       0.00195311])

In [10]:
mlp = MLP(Data.num_features,[Data.num_features,Data.num_features,Data.num_features],Data.num_classes)
mlp

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=784, bias=True)
    (1): Linear(in_features=784, out_features=784, bias=True)
    (2): Linear(in_features=784, out_features=784, bias=True)
    (3): Linear(in_features=784, out_features=10, bias=True)
  )
  (batch_norms): ModuleList(
    (0): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (dropout): Dropout(p=0.0, inplace=False)
)

In [11]:
logsoftmax = torch.nn.LogSoftmax(dim=1)
softmax = torch.nn.Softmax(dim=1)

logit  = mlp(train_x)
logp = logsoftmax(logit)
p = softmax(logit)
print(logp[:5],'\n\n',torch.exp(logp)[:5])

tensor([[ -81.1979,  -29.1538,  -94.3030,  -94.1308,  -41.4572, -103.1898,
            0.0000, -121.9277,  -69.8631,  -62.0304],
        [ -95.9643,  -25.5036, -126.4390, -124.1810,  -46.9770,  -96.6644,
            0.0000, -114.7425,  -80.5385,  -63.3069],
        [ -81.3244,  -33.7703, -147.5850,  -34.4398,  -55.0380, -108.2492,
            0.0000,  -56.5811,  -61.9946,  -42.3607],
        [ -57.8899,  -20.8471, -107.9544,  -74.1862,  -38.1200,  -75.4376,
            0.0000,  -88.0930,  -62.3543,  -74.1394],
        [ -68.9095,  -41.5605, -111.0749,  -99.9285,  -82.2764,  -78.3565,
            0.0000,  -56.1380,  -97.4619,  -95.8610]],
       grad_fn=<SliceBackward0>) 

 tensor([[5.4476e-36, 2.1810e-13, 1.1084e-41, 1.3168e-41, 9.8938e-19, 1.4013e-45,
         1.0000e+00, 0.0000e+00, 4.5585e-31, 1.1495e-27],
        [2.1048e-42, 8.3933e-12, 0.0000e+00, 0.0000e+00, 3.9641e-21, 1.0454e-42,
         1.0000e+00, 0.0000e+00, 1.0533e-35, 3.2075e-28],
        [4.8003e-36, 2.1564e-15, 0.0000e

In [12]:
print(Weak.M,'\n\n',Weak.z[:5],'\n\n',Weak.M[Weak.z][:5,:])

[[0.         0.         0.         ... 0.         0.00217014 0.00217014]
 [0.         0.         0.         ... 0.00217014 0.         0.00217014]
 [0.         0.         0.         ... 0.00217014 0.00217014 0.        ]
 ...
 [0.00195312 0.00195312 0.00195312 ... 0.00195312 0.         0.00195312]
 [0.00195312 0.00195312 0.00195312 ... 0.00195312 0.00195312 0.        ]
 [0.00195312 0.00195312 0.00195312 ... 0.00195312 0.00195312 0.00195312]] 

 tensor([ 79, 937, 474, 303, 554], dtype=torch.int32) 

 [[0.         0.         0.         0.00195312 0.         0.00195312
  0.         0.00195312 0.00195312 0.00195312]
 [0.00195312 0.00195312 0.00195312 0.         0.00195312 0.00195312
  0.         0.00195312 0.         0.        ]
 [0.         0.00195312 0.00195312 0.00195312 0.00195312 0.
  0.         0.00195312 0.         0.        ]
 [0.         0.00195312 0.         0.         0.00195312 0.00195312
  0.00195312 0.         0.         0.00195312]
 [0.00195312 0.         0.         0.        

In [13]:
#Im going to test the Partial loss
target = Weak.w[:5,:]/torch.sum(Weak.w[:5,:],dim=1,keepdim=True)


In [14]:
target[target>0]=1

In [15]:
rev = target*p[:5,:].clone().detach()
print(rev)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 1.3168e-41, 0.0000e+00, 1.4013e-45,
         0.0000e+00, 0.0000e+00, 4.5585e-31, 1.1495e-27],
        [2.1048e-42, 8.3933e-12, 0.0000e+00, 0.0000e+00, 3.9641e-21, 1.0454e-42,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 2.1564e-15, 0.0000e+00, 1.1040e-15, 1.2511e-24, 0.0000e+00,
         0.0000e+00, 2.6738e-25, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 8.8348e-10, 0.0000e+00, 0.0000e+00, 2.7840e-17, 1.7293e-33,
         1.0000e+00, 0.0000e+00, 0.0000e+00, 6.3337e-33],
        [1.1830e-30, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.8528e-36, 9.3373e-35,
         0.0000e+00, 4.1646e-25, 0.0000e+00, 2.3346e-42]], dtype=torch.float64)


In [16]:
s = rev.sum(dim=1).repeat(rev.size(1),1).transpose(0,1)
print(s)

tensor([[1.1500e-27, 1.1500e-27, 1.1500e-27, 1.1500e-27, 1.1500e-27, 1.1500e-27,
         1.1500e-27, 1.1500e-27, 1.1500e-27, 1.1500e-27],
        [8.3933e-12, 8.3933e-12, 8.3933e-12, 8.3933e-12, 8.3933e-12, 8.3933e-12,
         8.3933e-12, 8.3933e-12, 8.3933e-12, 8.3933e-12],
        [3.2604e-15, 3.2604e-15, 3.2604e-15, 3.2604e-15, 3.2604e-15, 3.2604e-15,
         3.2604e-15, 3.2604e-15, 3.2604e-15, 3.2604e-15],
        [1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00],
        [4.1646e-25, 4.1646e-25, 4.1646e-25, 4.1646e-25, 4.1646e-25, 4.1646e-25,
         4.1646e-25, 4.1646e-25, 4.1646e-25, 4.1646e-25]], dtype=torch.float64)


In [17]:
ts = torch.sum(rev,dim=1,keepdim=True)
print(ts)

tensor([[1.1500e-27],
        [8.3933e-12],
        [3.2604e-15],
        [1.0000e+00],
        [4.1646e-25]], dtype=torch.float64)


In [18]:
rev/ts

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 1.1450e-14, 0.0000e+00, 1.2185e-18,
         0.0000e+00, 0.0000e+00, 3.9640e-04, 9.9960e-01],
        [2.5077e-31, 1.0000e+00, 0.0000e+00, 0.0000e+00, 4.7229e-10, 1.2455e-31,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 6.6138e-01, 0.0000e+00, 3.3862e-01, 3.8372e-10, 0.0000e+00,
         0.0000e+00, 8.2008e-11, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 8.8348e-10, 0.0000e+00, 0.0000e+00, 2.7840e-17, 1.7293e-33,
         1.0000e+00, 0.0000e+00, 0.0000e+00, 6.3337e-33],
        [2.8407e-06, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.4489e-12, 2.2421e-10,
         0.0000e+00, 1.0000e+00, 0.0000e+00, 5.6058e-18]], dtype=torch.float64)

In [19]:
rev/s

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 1.1450e-14, 0.0000e+00, 1.2185e-18,
         0.0000e+00, 0.0000e+00, 3.9640e-04, 9.9960e-01],
        [2.5077e-31, 1.0000e+00, 0.0000e+00, 0.0000e+00, 4.7229e-10, 1.2455e-31,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 6.6138e-01, 0.0000e+00, 3.3862e-01, 3.8372e-10, 0.0000e+00,
         0.0000e+00, 8.2008e-11, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 8.8348e-10, 0.0000e+00, 0.0000e+00, 2.7840e-17, 1.7293e-33,
         1.0000e+00, 0.0000e+00, 0.0000e+00, 6.3337e-33],
        [2.8407e-06, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.4489e-12, 2.2421e-10,
         0.0000e+00, 1.0000e+00, 0.0000e+00, 5.6058e-18]], dtype=torch.float64)

In [20]:
Data.include_weak(w)

In [21]:
trainloader,testloader = Data.get_dataloader()

In [22]:
o,t,y,ind = next(iter(trainloader))

In [23]:
print(o, '\n\n', t, '\n\n', y,'\n\n', ind)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) 

 tensor([[1., 1., 1., 1., 1., 1., 1., 0., 1., 1.],
        [0., 1., 1., 0., 1., 0., 1., 1., 0., 0.],
        [1., 0., 1., 1., 1., 0., 1., 0., 0., 0.],
        [1., 1., 1., 0., 1., 1., 0., 0., 1., 1.],
        [0., 0., 1., 0., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 1., 0., 1., 0., 1.],
        [0., 1., 1., 0., 0., 0., 0., 1., 0., 1.]], dtype=torch.float64) 

 tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0

In [24]:
trainloader.dataset.tensors[1][[59922, 28394, 16042, 33821, 28968, 51396, 42273, 56490]]

tensor([[0., 1., 0., 0., 1., 1., 0., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0., 1., 0., 1., 0.],
        [0., 1., 0., 0., 1., 0., 0., 1., 1., 1.],
        [1., 0., 1., 1., 0., 1., 1., 0., 0., 1.],
        [1., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 1.],
        [1., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 1., 0., 1., 1., 0., 1.]], dtype=torch.float64)

In [25]:
#optim = torch.optim.SGD(list(mlp.parameters()),lr = 1e-2, weight_decay = 1e-4, momentum = 0.9)
#optim = torch.optim.Adam(mlp.parameters(),lr = 1e-2)

#loss = PartialLoss(trainloader.dataset.tensors[1])

In [26]:
#mlp,results = train_and_evaluate(mlp,trainloader,testloader,optim,loss,num_epochs=10)

In [27]:
list(mlp.parameters())

[Parameter containing:
 tensor([[ 0.0210,  0.0098, -0.0363,  ..., -0.0449, -0.0424,  0.0076],
         [-0.0354,  0.0375, -0.0562,  ...,  0.0552, -0.0114, -0.0444],
         [ 0.0339,  0.0040,  0.0408,  ..., -0.0582,  0.0558, -0.0314],
         ...,
         [ 0.0172, -0.0480, -0.0061,  ...,  0.0089, -0.0298, -0.0208],
         [-0.0165, -0.0182, -0.0461,  ...,  0.0006, -0.0058,  0.0562],
         [-0.0195,  0.0265, -0.0231,  ..., -0.0438, -0.0348,  0.0196]],
        requires_grad=True),
 Parameter containing:
 tensor([ 2.4323e-02, -2.9067e-02, -4.6438e-03,  2.8989e-02, -2.2289e-02,
         -4.3756e-03,  3.1826e-02, -3.0761e-02,  2.2375e-03,  1.9557e-02,
          1.2819e-03, -1.4787e-02, -8.0491e-03,  1.7845e-02, -1.9567e-02,
         -3.4515e-02, -2.5450e-02,  2.4920e-02, -3.5144e-02,  3.2672e-02,
         -1.2957e-02,  2.3730e-02,  2.1464e-02,  7.9114e-03,  3.4305e-02,
         -1.1410e-02, -2.9093e-02,  1.9956e-02,  2.6921e-02,  3.1814e-02,
         -1.7220e-02,  6.5969e-03, -1.04

In [28]:
mlp,results = warm_up(mlp,trainloader,testloader,num_epochs=10)

Epoch 1/10: Train Loss: 3.1042, Train Acc: 0.1124, Test Acc: 0.1135
Epoch 2/10: Train Loss: nan, Train Acc: 0.0987, Test Acc: 0.0980
Epoch 3/10: Train Loss: nan, Train Acc: 0.0987, Test Acc: 0.0980
Epoch 4/10: Train Loss: nan, Train Acc: 0.0987, Test Acc: 0.0980


KeyboardInterrupt: 